In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install lion-pytorch

In [ ]:
import cv2
import pywt
import pywt.data
import torch
import numpy as np
from skimage import exposure
from skimage.filters import threshold_otsu
from lion_pytorch import Lion  # You might need to implement or find a suitable LOA library

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Reshape


# Step 1: Pre-processing with a combination of wavelet and Gaussian filters
def pre_process_image(image):
    if len(image.shape) == 3:  # Check if the image is color
        # Convert the image to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image

    # Apply wavelet filter (you can choose a specific wavelet)
    coeffs = pywt.dwt2(gray_image, 'bior1.3')
    wavelet_image = pywt.idwt2(coeffs, 'bior1.3')

    # Resize the wavelet image to match the dimensions of the original image
    wavelet_image = cv2.resize(wavelet_image, (image.shape[1], image.shape[0]))

    if len(image.shape) == 3:  # Check if the image is color
        # Apply Gaussian filter separately for each channel
        gaussian_image = np.zeros_like(image, dtype=np.float32)
        for i in range(image.shape[2]):
            gaussian_image[:, :, i] = cv2.GaussianBlur(image[:, :, i].astype(np.float32), (5, 5), 0)
    else:
        # Apply Gaussian filter for grayscale images
        gaussian_image = cv2.GaussianBlur(image.astype(np.float32), (5, 5), 0)

    # Ensure both images have the same shape before addition
    if len(image.shape) == 3:  # Check if the image is color
        # Convert the grayscale image to a color image with identical channels
        wavelet_image = cv2.cvtColor(wavelet_image.astype(np.uint8), cv2.COLOR_GRAY2BGR)

    # Combine the two filtered images (you can adjust the weights)
    hybrid_image = 0.7 * wavelet_image.astype(np.float32) + 0.3 * gaussian_image

    return hybrid_image

# Step 2: Segmentation using Otsu thresholding
def segmentation(image):
    # Convert to 8-bit image
    image_8bit = cv2.convertScaleAbs(image)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image_8bit, cv2.COLOR_BGR2GRAY)

    # Apply Otsu thresholding
    _, segmented_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return segmented_image

# Step 3: Feature extraction using Lion Optimization Algorithm
def lion_optimization_algorithm(data):
    data_tensor = torch.tensor(data, dtype=torch.float32)
    
    # Create an instance of Lion optimizer
    loa_optimizer = Lion([{'params': data_tensor}], lr=1e-4, weight_decay=1e-2)

    # You may need to adjust the number of optimization steps based on your requirements
    num_optimization_steps = 100

    # Perform optimization steps
    for _ in range(num_optimization_steps):
        # Assuming you have a closure function for computing the loss
        def closure():
            # Your computation of loss goes here
            loss = compute_loss(data_tensor)
            return loss

        loa_optimizer.step(closure)

    # Get the optimized features (assuming LOA modifies the input data in-place)
    optimized_features = data_tensor.numpy()

    return optimized_features

def compute_loss(data_tensor):
    # Replace this with your actual computation of loss based on the data_tensor
    # For example, you might have a machine learning model, and this could be the forward pass
    # Make sure to return a torch.Tensor representing the loss
    loss = torch.tensor(0.0, requires_grad=True)
    return loss

In [ ]:
normal_files=os.listdir('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL')

print(normal_files[0:5])
print(normal_files[-5:])

In [ ]:
Pneumonia_files=os.listdir('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA')

print(Pneumonia_files[0:5])
print(Pneumonia_files[-5:])

In [ ]:
print('Normal: ', len(normal_files))
print('Pneumonia: ',len(Pneumonia_files))

In [ ]:
Normal_label=[0]*1341
Pneumonia_label=[1]*3875

In [ ]:
print('With Normal labels: ', Normal_label[0:5])
print('Without Pneumonia labels: ', Pneumonia_label[0:5])

In [ ]:
labels=Normal_label + Pneumonia_label

print('Labels are: ',len(labels))

print(labels[0:5])
print(labels[-5:])

In [ ]:
from PIL import Image

normal_path=('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/')
data=[]

for img_file in normal_files:
    image=Image.open(normal_path +img_file)
    image=image.resize((128,128))
    image=image.convert('RGB')
    image=np.array(image)
    data.append(image)
    
Pneumonia_path=('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/')

for img_file in Pneumonia_files:
    image=Image.open(Pneumonia_path + img_file )
    image=image.resize((128,128))
    image=image.convert('RGB')
    image=np.array(image)
    data.append(image)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense, Reshape

def cnn_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Reshape((-1, 64)))
    model.add(LSTM(100))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
from keras.utils import to_categorical
# Example Usage:
# Load your image data
#image_data = cv2.imread('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL/IM-0115-0001.jpeg')

# Step 1: Pre-processing
#pre_processed_image = pre_process_image(data)
pre_processed_train_data = np.array([pre_process_image(img) for img in data])


# Step 2: Segmentation
#segmented_image = segmentation(pre_processed_image)
segmented_train_data = np.array([segmentation(img) for img in pre_processed_train_data])


# Step 3: Feature extraction
#extracted_features = lion_optimization_algorithm(segmented_image)
optimized_train_features = lion_optimization_algorithm(segmented_train_data)
# Assuming optimized_train_features has shape (batch_size, height, width)
optimized_train_features = np.array([lion_optimization_algorithm(segmented_img) for segmented_img in segmented_train_data])


# Expand dimensions to include the channel
optimized_train_features = np.expand_dims(optimized_train_features, axis=-1)


# Verify the shape after modification
print(optimized_train_features.shape)

In [ ]:
# Determine the sequence length and number of features
#sequence_length = optimized_train_features.shape[1]  # Adjust this based on your data
#num_features = optimized_train_features.shape[2]  # Adjust this based on your data
# Reshape the data to have the required 3D shape
#reshaped_features = optimized_train_features.reshape(-1, sequence_length, num_features)


# Step 4: Categorization using CNN-LSTM
input_shape = (128, 128, 3)  # Assuming RGB images
num_classes = 2  # Specify the number of classes in your dataset

#model = cnn_lstm_model(input_shape, num_classes)

# Convert labels to one-hot encoding
train_labels_one_hot = to_categorical(labels, num_classes)

# Create the model
model = cnn_lstm_model(input_shape, num_classes)
#model.add(LSTM(100, input_shape=(sequence_length, num_features)))
# Train the model
model.fit(optimized_train_features, train_labels_one_hot, epochs=10, batch_size=32, validation_split=0.2)